# Project 6: Analyzing Stock Sentiment from Twits
## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

### Load Packages

In [1]:
import json
import nltk
import os
import random
import re
import torch

from torch import nn, optim
import torch.nn.functional as F

## Introduction
When deciding the value of a company, it's important to follow the news. For example, a product recall or natural disaster in a company's product chain. You want to be able to turn this information into a signal. Currently, the best tool for the job is a Neural Network. 

For this project, you'll use posts from the social media site [StockTwits](https://en.wikipedia.org/wiki/StockTwits). The community on StockTwits is full of investors, traders, and entrepreneurs. Each message posted is called a Twit. This is similar to Twitter's version of a post, called a Tweet. You'll build a model around these twits that generate a sentiment score.

We've collected a bunch of twits, then hand labeled the sentiment of each. To capture the degree of sentiment, we'll use a five-point scale: very negative, negative, neutral, positive, very positive. Each twit is labeled -2 to 2 in steps of 1, from very negative to very positive respectively. You'll build a sentiment analysis model that will learn to assign sentiment to twits on its own, using this labeled data.

The first thing we should to do, is load the data.

## Import Twits 
### Load Twits Data 
This JSON file contains a list of objects for each twit in the `'data'` field:

```
{'data':
  {'message_body': 'Neutral twit body text here',
   'sentiment': 0},
  {'message_body': 'Happy twit body text here',
   'sentiment': 1},
   ...
}
```

The fields represent the following:

* `'message_body'`: The text of the twit.
* `'sentiment'`: Sentiment score for the twit, ranges from -2 to 2 in steps of 1, with 0 being neutral.


To see what the data look like by printing the first 10 twits from the list. 

In [2]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'twits.json'), 'r') as f:
    twits = json.load(f)

print(twits['data'][:10])

[{'message_body': '$FITB great buy at 26.00...ill wait', 'sentiment': 2, 'timestamp': '2018-07-01T00:00:09Z'}, {'message_body': '@StockTwits $MSFT', 'sentiment': 1, 'timestamp': '2018-07-01T00:00:42Z'}, {'message_body': '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', 'sentiment': 2, 'timestamp': '2018-07-01T00:01:24Z'}, {'message_body': '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', 'sentiment': 1, 'timestamp': '2018-07-01T00:01:47Z'}, {'message_body': '$AMD reveal yourself!', 'sentiment': 0, 'timestamp': '2018-07-01T00:02:13Z'}, {'message_body': '$AAPL Why the drop? I warren Buffet taking out his position?', 'sentiment': 1, 'timestamp': '2018-07-01T00:03:10Z'}, {'message_body': '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=BA', 'sentiment': -2, 'timestamp': '2018-07-01T

In [3]:
twits['data'][:2]

[{'message_body': '$FITB great buy at 26.00...ill wait',
  'sentiment': 2,
  'timestamp': '2018-07-01T00:00:09Z'},
 {'message_body': '@StockTwits $MSFT',
  'sentiment': 1,
  'timestamp': '2018-07-01T00:00:42Z'}]

### Length of Data
Now let's look at the number of twits in dataset. Print the number of twits below.

In [4]:
"""print out the number of twits"""

# TODO Implement 
len(twits['data'])

1548010

### Split Message Body and Sentiment Score

In [5]:
messages = [twit['message_body'] for twit in twits['data']]
# Since the sentiment scores are discrete, we'll scale the sentiments to 0 to 4 for use in our network
sentiments = [twit['sentiment'] + 2 for twit in twits['data']]

## Preprocessing the Data
With our data in hand we need to preprocess our text. These twits are collected by filtering on ticker symbols where these are denoted with a leader $ symbol in the twit itself. For example,

`{'message_body': 'RT @google Our annual look at the year in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG',
 'sentiment': 0}`

The ticker symbols don't provide information on the sentiment, and they are in every twit, so we should remove them. This twit also has the `@google` username, again not providing sentiment information, so we should also remove it. We also see a URL `http://t.co/sptHOAh8`. Let's remove these too.

The easiest way to remove specific words or phrases is with regex using the `re` module. You can sub out specific patterns with a space:

```python
re.sub(pattern, ' ', text)
```
This will substitute a space with anywhere the pattern matches in the text. Later when we tokenize the text, we'll split appropriately on those spaces.

### Pre-Processing

In [6]:
nltk.download('wordnet')


def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - tokenize by splitting the string on whitespace 
        - removes any single character tokens
    
    Parameters
    ----------
        message : The text message to be preprocessed.
        
    Returns
    -------
        tokens: The preprocessed text into tokens.
    """ 
    #TODO: Implement 
    
    # Lowercase the twit message
    text = message.lower()
    
#     # Replace URLs with a space in the message
    text = re.sub(r'http\S+', ' ', text)
    
#     # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    text = re.sub(r'\$\S+', '', text)
    
#     # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    text = re.sub(r'@\S+', ' ', text)

#     # Replace everything not a letter with a space
    text = re.sub(r'[^A-Za-z]', ' ', text)
    
#     # Tokenize by splitting the string on whitespace into a list of words
    tokens = text.split()

    # Lemmatize words using the WordNetLemmatizer. You can ignore any word that is not longer than one character.
    wnl = nltk.stem.WordNetLemmatizer()
    tokens = [wnl.lemmatize(w) for w in tokens if len(w) > 1]
    return tokens
#     return tokens

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [7]:
print(messages[104])

$HBAN short interest ratio is 2.23 on 2018-06-15 and Short % to float is 1.76% http://sunshineavenue.com/stock/HBAN/ via @sunshineave


In [8]:
print(preprocess(messages[104]))

['short', 'interest', 'ratio', 'is', 'on', 'and', 'short', 'to', 'float', 'is', 'via']


### Preprocess All the Twits 
Now we can preprocess each of the twits in our dataset. Apply the function `preprocess` to all the twit messages.

In [9]:
# TODO Implement

tokenized = [preprocess(m) for m in messages]

In [10]:
print(len(tokenized))
print(tokenized[104])

1548010
['short', 'interest', 'ratio', 'is', 'on', 'and', 'short', 'to', 'float', 'is', 'via']


In [11]:
tokenized[:3]

[['great', 'buy', 'at', 'ill', 'wait'],
 [],
 ['staanalystalert',
  'for',
  'jefferies',
  'maintains',
  'with',
  'rating',
  'of',
  'hold',
  'setting',
  'target',
  'price',
  'at',
  'usd',
  'our',
  'own',
  'verdict',
  'is',
  'buy']]

In [12]:
# tokenized = [l for l in tokenized if len(l) > 0] 
# len(tokenized)

### Bag of Words
Now with all of our messages tokenized, we want to create a vocabulary and count up how often each word appears in our entire corpus. Use the [`Counter`](https://docs.python.org/3.1/library/collections.html#collections.Counter) function to count up all the tokens.

In [13]:
from collections import Counter


"""
Create a vocabulary by using Bag of words
"""

# TODO: Implement 

bow = Counter([l for ll in tokenized for l in ll])

In [14]:
print(len(bow))
print(bow['apple'])

98368
12826


In [15]:
bow.most_common(5)

[('the', 398753),
 ('to', 379487),
 ('is', 284865),
 ('for', 273537),
 ('on', 241663)]

### Frequency of Words Appearing in Message
With our vocabulary, now we'll remove some of the most common words such as 'the', 'and', 'it', etc. These words don't contribute to identifying sentiment and are really common, resulting in a lot of noise in our input. If we can filter these out, then our network should have an easier time learning.

We also want to remove really rare words that show up in a only a few twits. Here you'll want to divide the count of each word by the number of messages. Then remove words that only appear in some small fraction of the messages.

In [16]:
"""
Set the following variables:
    freqs
    low_cutoff
    high_cutoff
    K_most_common
"""

# TODO Implement 
n_twits = len(tokenized)
n_words = len(bow)

# Dictionart that contains the Frequency of words appearing in messages.
# The key is the token and the value is the frequency of that word in the corpus.

freqs = {k:v/n_words for k,v in bow.items()}

# Float that is the frequency cutoff. Drop words with a frequency that is lower or equal to this number.

# print(len([w for w,v in bow.items() if v/n_words < 0.00002]))
low_cutoff = 5e-5

# Integer that is the cut off for most common words. Drop words that are the `high_cutoff` most common words.
high_cutoff = 20

# The k most common words in the corpus. Use `high_cutoff` as the k.
K_most_common = [w[0] for w in bow.most_common(n=high_cutoff)]


filtered_words = [word for word in freqs if (freqs[word] > low_cutoff and word not in K_most_common)]
print(K_most_common)
len(filtered_words) 

['the', 'to', 'is', 'for', 'on', 'of', 'and', 'in', 'this', 'it', 'at', 'will', 'up', 'are', 'you', 'that', 'be', 'short', 'what', 'today']


27401

In [17]:
print(filtered_words[-10:])
print(filtered_words[:10])

['wrd', 'cahill', 'sabelhaus', 'elwyn', 'tashtego', 'auerbach', 'donahoe', 'xforecaster', 'sherry', 'alwayswrong']
['great', 'buy', 'ill', 'wait', 'staanalystalert', 'jefferies', 'maintains', 'with', 'rating', 'hold']


In [18]:
# bow.most_common(n=25)
# sorted(bow.values(), reverse=False)

# len([w for w,v in bow.items() if v < 2])
# len([w for w,v in bow.items() if v/n_twits < 0.000002])

### Updating Vocabulary by Removing Filtered Words
Let's creat three variables that will help with our vocabulary.

In [19]:
"""
Set the following variables:
    vocab
    id2vocab
    filtered
"""
#TODO Implement

# A dictionary for the `filtered_words`. The key is the word and value is an id that represents the word. 
vocab = {w:idx for idx,w in enumerate(filtered_words)}
# Reverse of the `vocab` dictionary. The key is word id and value is the word. 
id2vocab = {v:k for k,v in vocab.items()}
# tokenized with the words not in `filtered_words` removed.
# filtered = []
# for w in tokenized:
#     filtered.append([word for word in w if word not in filtered_words])
# filtered = [word for word in [w for w in tokenized] if word in filtered_words]
filtered = [[w for w in token if w in vocab] for token in tokenized]


In [20]:
len(filtered)

1548010

In [21]:
print(vocab['apple'])
print(id2vocab[vocab['apple']])

591
apple


### Balancing the classes
Let's do a few last pre-processing steps. If we look at how our twits are labeled, we'll find that 50% of them are neutral. This means that our network will be 50% accurate just by guessing 0 every single time. To help our network learn appropriately, we'll want to balance our classes.
That is, make sure each of our different sentiment scores show up roughly as frequently in the data.

What we can do here is go through each of our examples and randomly drop twits with neutral sentiment. What should be the probability we drop these twits if we want to get around 20% neutral twits starting at 50% neutral? We should also take this opportunity to remove messages with length 0.

In [22]:
balanced = {'messages': [], 'sentiments':[]}

n_neutral = sum(1 for each in sentiments if each == 2)
N_examples = len(sentiments)
keep_prob = (N_examples - n_neutral)/4/n_neutral

for idx, sentiment in enumerate(sentiments):
    message = filtered[idx]
    if len(message) == 0:
        # skip this message because it has length zero
        continue
    elif sentiment != 2 or random.random() < keep_prob:
        balanced['messages'].append(message)
        balanced['sentiments'].append(sentiment) 

If you did it correctly, you should see the following result 

In [23]:
n_neutral = sum(1 for each in balanced['sentiments'] if each == 2)
N_examples = len(balanced['sentiments'])
n_neutral/N_examples

0.1948397270435519

Finally let's convert our tokens into integer ids which we can pass to the network.

In [24]:
token_ids = [[vocab[word] for word in message] for message in balanced['messages']]
sentiments = balanced['sentiments']

In [25]:
train_on_gpu=torch.cuda.is_available()
train_on_gpu

True

## Neural Network
Now we have our vocabulary which means we can transform our tokens into ids, which are then passed to our network. So, let's define the network now!

Here is a nice diagram showing the network we'd like to build: 

#### Embed -> RNN -> Dense -> Softmax
### Implement the text classifier
Before we build text classifier, if you remember from the other network that you built in  "Sentiment Analysis with an RNN"  exercise  - which there, the network called " SentimentRNN", here we named it "TextClassifer" - consists of three main parts: 1) init function `__init__` 2) forward pass `forward`  3) hidden state `init_hidden`. 

This network is pretty similar to the network you built expect in the  `forward` pass, we use softmax instead of sigmoid. The reason we are not using sigmoid is that the output of NN is not a binary. In our network, sentiment scores have 5 possible outcomes. We are looking for an outcome with the highest probability thus softmax is a better choice.

In [26]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, lstm_size, output_size, lstm_layers=1, dropout=0.1):
        """
        Initialize the model by setting up the layers.
        
        Parameters
        ----------
            vocab_size : The vocabulary size.
            embed_size : The embedding layer size.
            lstm_size : The LSTM layer size.
            output_size : The output size.
            lstm_layers : The number of LSTM layers.
            dropout : The dropout probability.
        """
        
    
        """
        Initialize the mode by setting up the layers
        input layer = vocab_size
        embed layer = embed_size
        hidden layer = lstm_size
        number of hidden layers - lstm_layers
        output layer = output_size
        """
        
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.lstm_size = lstm_size
        self.output_size = output_size
        self.lstm_layers = lstm_layers
        self.dropout = dropout
        
        # TODO Implement

        # Setup embedding layer
        self.embedding = nn.Embedding(num_embeddings=self.vocab_size,
                                      embedding_dim= self.embed_size)
        
        
        # Setup additional layers
        # Hidden Layer
        self.lstm = nn.LSTM(self.embed_size, 
                           self.lstm_size, 
                           self.lstm_layers,
                           batch_first=False,
                           dropout=self.dropout)
        
        
        # Dropout Layer
        self.dropout = nn.Dropout(0.3)
        
        # linear and softmax layer
        self.fc = nn.Linear(self.lstm_size, self.output_size)
        self.logmax = nn.LogSoftmax(dim=1)


    def init_hidden(self, batch_size):
        """ 
        Initializes hidden state
        
        Parameters
        ----------
            batch_size : The size of batches.
        
        Returns
        -------
            hidden_state
            
        """
        
        # TODO Implement 
        
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
#         if True:
#             hidden_state = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_().cuda(),
#                   weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_().cuda())
# #         if (train_on_gpu):
# #         hidden_state = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_().cuda(), weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_().cuda())
#         else:
        hidden_state = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_(),
                            weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_())
        
        return hidden_state


    def forward(self, nn_input, hidden_state):
        """
        Perform a forward pass of our model on nn_input.
        
        Parameters
        ----------
            nn_input : The batch of input to the NN.
            hidden_state : The LSTM hidden state.

        Returns
        -------
            logps: log softmax output
            hidden_state: The new hidden state.

        """
        # TODO Implement
#         batch_size = nn_input.size(0)
        
#         nn_input = nn_input.long()
        embeds = self.embedding(nn_input)
        lstm_out, hidden_state = self.lstm(embeds, hidden_state)
        lstm_out = lstm_out[-1,:,:]
#         lstm_out = lstm_out.contiguous().view(-1, self.lstm_size)

        #stack up lstm outputs
        out = self.dropout(lstm_out)
        out = self.fc(out)
        logps = self.logmax(out)
        
        # reshape to be batch_size first
#         logps = logps.view(batch_size, -1)
#         logps = logps[:, -1]
         
        
        return logps, hidden_state

### View Model

In [27]:
model = TextClassifier(len(vocab), 10, 6, 5, dropout=0.1, lstm_layers=2)
model.embedding.weight.data.uniform_(-1, 1)
input = torch.randint(0, 1000, (5, 4), dtype=torch.int64)
hidden = model.init_hidden(4)

logps, _ = model.forward(input, hidden)
print(logps)

tensor([[-1.7810, -1.5911, -1.5683, -1.5607, -1.5634],
        [-1.8105, -1.6740, -1.6470, -1.5719, -1.3917],
        [-1.7890, -1.5632, -1.5840, -1.5768, -1.5530],
        [-1.7916, -1.5631, -1.5848, -1.5772, -1.5498]])


## Training
### DataLoaders and Batching
Now we should build a generator that we can use to loop through our data. It'll be more efficient if we can pass our sequences in as batches. Our input tensors should look like `(sequence_length, batch_size)`. So if our sequences are 40 tokens long and we pass in 25 sequences, then we'd have an input size of `(40, 25)`.

If we set our sequence length to 40, what do we do with messages that are more or less than 40 tokens? For messages with fewer than 40 tokens, we will pad the empty spots with zeros. We should be sure to **left** pad so that the RNN starts from nothing before going through the data. If the message has 20 tokens, then the first 20 spots of our 40 long sequence will be 0. If a message has more than 40 tokens, we'll just keep the first 40 tokens.

In [28]:
def dataloader(messages, labels, sequence_length=30, batch_size=32, shuffle=False):
    """ 
    Build a dataloader.
    """
    if shuffle:
        indices = list(range(len(messages)))
        random.shuffle(indices)
        messages = [messages[idx] for idx in indices]
        labels = [labels[idx] for idx in indices]

    total_sequences = len(messages)

    for ii in range(0, total_sequences, batch_size):
        batch_messages = messages[ii: ii+batch_size]
        
        # First initialize a tensor of all zeros
        batch = torch.zeros((sequence_length, len(batch_messages)), dtype=torch.int64)
        for batch_num, tokens in enumerate(batch_messages):
            token_tensor = torch.tensor(tokens)
            # Left pad!
            start_idx = max(sequence_length - len(token_tensor), 0)
            batch[start_idx:, batch_num] = token_tensor[:sequence_length]
        
        label_tensor = torch.tensor(labels[ii: ii+len(batch_messages)])
        
        yield batch, label_tensor

### Training and  Validation
With our data in nice shape, we'll split it into training and validation sets.

In [29]:
"""
Split data into training and validation datasets. Use an appropriate split size.
The features are the `token_ids` and the labels are the `sentiments`.
"""   

# TODO Implement 
split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)

split_idx = int(len(token_ids)*split_frac)
train_features, valid_features = token_ids[:split_idx], token_ids[split_idx:]
train_labels, valid_labels = sentiments[:split_idx], sentiments[split_idx:]

## print out the shapes of your resultant feature data

# train_features = pass
# valid_features = pass
# train_labels = pass
# valid_labels = pass

In [30]:
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=20, batch_size=64)))
model = TextClassifier(len(vocab)+1, 200, 128, 5, dropout=0.)
hidden = model.init_hidden(64)
logps, hidden = model.forward(text_batch, hidden)

### Training
It's time to train the neural network!

In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TextClassifier(len(vocab)+1, 1024, 512, 5, lstm_layers=2, dropout=0.2)
model.embedding.weight.data.uniform_(-1, 1)
model.to(device)

TextClassifier(
  (embedding): Embedding(27402, 1024)
  (lstm): LSTM(1024, 512, num_layers=2, dropout=0.2)
  (dropout): Dropout(p=0.3)
  (fc): Linear(in_features=512, out_features=5, bias=True)
  (logmax): LogSoftmax()
)

In [32]:
device

device(type='cuda')

In [33]:
"""
Train your model with dropout. Make sure to clip your gradients.
Print the training loss, validation loss, and validation accuracy for every 100 steps.
"""
import numpy as np

epochs = 3
batch_size = 512
learning_rate = 0.001

clip=5 # gradient clipping

print_every = 100
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model.train()

for epoch in range(epochs):
    print('Starting epoch {}'.format(epoch + 1))
    
    steps = 0
    for text_batch, labels in dataloader(train_features, train_labels, batch_size=batch_size):
        steps += 1
         # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        
        hidden = model.init_hidden(labels.shape[0])
        
        # Set Device
        text_batch, labels = text_batch.to(device), labels.to(device)
        
        for each in hidden:
            each.to(device)
        
        # TODO Implement: Train Model
        model.zero_grad()
        
        output, hidden = model(text_batch, hidden)     
        
        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels)
        loss.backward()       
        
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()          
        
        if steps % print_every == 0:
            model.eval()
            
            val_losses = []
            for inputs, labels in dataloader(valid_features, valid_labels, batch_size=batch_size):

                #val_h = tuple([each.data for each in val_h])

                inputs, labels = inputs.to(device), labels.to(device)
                
                val_h = model.init_hidden(labels.shape[0])
                for each in val_h:
                    each.to(device)                

                output, val_h = model(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels)

                val_losses.append(val_loss.item())            
            # TODO Implement: Print metrics
            
            model.train()
            print("Epoch: {}/{}...".format(epoch+1, epochs),
                  "Step: {}...".format(steps),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Starting epoch 1
Epoch: 1/3... Step: 100... Loss: 1.097647... Val Loss: 1.123254
Epoch: 1/3... Step: 200... Loss: 0.539070... Val Loss: 1.049999
Epoch: 1/3... Step: 300... Loss: 0.835744... Val Loss: 0.968889
Epoch: 1/3... Step: 400... Loss: 0.837163... Val Loss: 0.957342
Epoch: 1/3... Step: 500... Loss: 0.901913... Val Loss: 0.924827
Epoch: 1/3... Step: 600... Loss: 0.584802... Val Loss: 0.901477
Epoch: 1/3... Step: 700... Loss: 0.679595... Val Loss: 0.873802
Epoch: 1/3... Step: 800... Loss: 0.897964... Val Loss: 0.890255
Epoch: 1/3... Step: 900... Loss: 1.015017... Val Loss: 0.882188
Epoch: 1/3... Step: 1000... Loss: 0.735952... Val Loss: 0.877833
Epoch: 1/3... Step: 1100... Loss: 0.888960... Val Loss: 0.859980
Epoch: 1/3... Step: 1200... Loss: 0.809891... Val Loss: 0.873717
Epoch: 1/3... Step: 1300... Loss: 0.768291... Val Loss: 0.857948
Epoch: 1/3... Step: 1400... Loss: 0.810770... Val Loss: 0.846287
Epoch: 1/3... Step: 1500... Loss: 0.848268... Val Loss: 0.838238
Epoch: 1/3... Ste

## Making Predictions
### Prediction 
Okay, now that you have a trained model, try it on some new twits and see if it works appropriately. Remember that for any new text, you'll need to preprocess it first before passing it to the network. Implement the `predict` function to generate the prediction vector from a message.

In [35]:
len(vocab)

27401

In [42]:
def predict(text, model, vocab):
    """ 
    Make a prediction on a single sentence.

    Parameters
    ----------
        text : The string to make a prediction on.
        model : The model to use for making the prediction.
        vocab : Dictionary for word to word ids. The key is the word and the value is the word id.

    Returns
    -------
        pred : Prediction vector
    """    
    
    # TODO Implement
    
    tokens = preprocess(text)
    
    # Filter non-vocab words
    tokens = [word for word in tokens if word in vocab]
    # Convert words to ids
    tokens = [vocab[word] for word in tokens]
        
    # Adding a batch dimension
    text_input = torch.tensor(tokens).unsqueeze(1)
    # Get the NN output
    hidden = model.init_hidden(1)
    logps, _ = model.forward(text_input, hidden)
    # Take the exponent of the NN output to get a range of 0 to 1 for each label.
    pred = torch.exp(logps)
    
    return pred

In [43]:
text = "Google is working on self driving cars, I'm bullish on $goog"
model.eval()
model.to("cpu")
predict(text, model, vocab)

tensor([[ 0.0002,  0.0071,  0.0042,  0.7120,  0.2766]])

### Questions: What is the prediction of the model? What is the uncertainty of the prediction?

We have five classes for output ranging from 0-4 (originally -2 to 2 where 0 is neutral). 0 is very negative, 1 is negative, 2 is neutral, 3 is positive, and 4 being very positive.

In this case the class predicted is 1 with 70% likelihood (highest among the other classes). Class 1 represents "negative" sentiment.

In [54]:
p = predict(text, model, vocab)
print("Class = ", np.argmax(p.detach().numpy())-2)
print("Percent Prob = ", np.max(p.detach().numpy()))

Class =  1
Percent Prob =  0.711991


Now we have a trained model and we can make predictions. We can use this model to track the sentiments of various stocks by predicting the sentiments of twits as they are coming in. Now we have a stream of twits. For each of those twits, pull out the stocks mentioned in them and keep track of the sentiments. Remember that in the twits, ticker symbols are encoded with a dollar sign as the first character, all caps, and 2-4 letters, like $AAPL. Ideally, you'd want to track the sentiments of the stocks in your universe and use this as a signal in your larger model(s).

## Testing
### Load the Data 

In [55]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'test_twits.json'), 'r') as f:
    test_data = json.load(f)

### Twit Stream

In [56]:
def twit_stream():
    for twit in test_data['data']:
        yield twit

next(twit_stream())

{'message_body': '$JWN has moved -1.69% on 10-31. Check out the movement and peers at  https://dividendbot.com?s=JWN',
 'timestamp': '2018-11-01T00:00:05Z'}

Using the `prediction` function, let's apply it to a stream of twits.

In [64]:
def score_twits(stream, model, vocab, universe):
    """ 
    Given a stream of twits and a universe of tickers, return sentiment scores for tickers in the universe.
    """
    for twit in stream:
#         print(twit)

        # Get the message text
        text = twit['message_body']
        symbols = re.findall('\$[A-Z]{2,4}', text)
        score = predict(text, model, vocab)

        for symbol in symbols:
            if symbol in universe:
                yield {'symbol': symbol, 'score': score, 'timestamp': twit['timestamp']}

In [65]:
universe = {'$BBRY', '$AAPL', '$AMZN', '$BABA', '$YHOO', '$LQMT', '$FB', '$GOOG', '$BBBY', '$JNUG', '$SBUX', '$MU'}
score_stream = score_twits(twit_stream(), model, vocab, universe)

next(score_stream)

{'symbol': '$AAPL',
 'score': tensor([[ 0.0539,  0.0959,  0.1114,  0.2492,  0.4897]]),
 'timestamp': '2018-11-01T00:00:18Z'}

That's it. You have successfully built a model for sentiment analysis! 

## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.